In [2]:
%load_ext autoreload
%autoreload 2

In [40]:
# !pip install folium pandas python-dotenv

## Searching for comparables

#### /api/integration/v2/comparables/search/advanced
https://docs.urbandataanalytics.com/#tag/Comparables

In [1]:
from utils import Search

search = Search().request(
    address='Av de Brasil 4, madrid',
    typology_type_id=1,
    property_type_id=4,
    is_active=0,
    sort_by='distance',
    ascending=1,
)

search.map()

Total: 280 - Page: 1/10


In [3]:
search.df[:1].to_dict()

{'_id': {0: '66-1176540468'},
 'title': {0: 'Piso en venta en Poeta Joan Maragall Cuatro Caminos - Azca'},
 'is_active': {0: 0},
 'action_id': {0: 3},
 'action_name': {0: 'Unpublished w/o confirmed date'},
 'usage_id': {0: 1},
 'usage_name': {0: 'Residential'},
 'comparison_type': {0: 0},
 'operation_type_id': {0: 1},
 'operation_type_name': {0: 'Venta'},
 'last_price': {0: 825000.0},
 'last_pm2': {0: 5093.0},
 'address': {0: 'Poeta Joan Maragall, Cuatro Caminos - Azca'},
 'location': {0: {'coordinates': [-3.6928477, 40.453594], 'type': 'Point'}},
 'distance': {0: 20},
 'similarity': {0: 2},
 'property_type_id': {0: 4},
 'property_type_name': {0: 'Apartment'},
 'subtypology_name': {0: 'Colective housing'},
 'subtypology_type_id': {0: 2},
 'typology_type_id': {0: 1},
 'typology_type_name': {0: 'Residencial'},
 'construction_type_id': {0: 2},
 'construction_type_name': {0: 'Segunda mano'},
 'build_status_id': {0: 0},
 'build_status_name': {0: 'Desconocido'},
 'date_in': {0: {'$date': '20

In [4]:
from utils import show_images

show_images(search.df.loc[:3])

,_id,title,url,img1,img2,img3
0,66-1176540468,Piso en venta en Poeta Joan Maragall Cuatro Caminos - Azca,https://www.fotocasa . 76540468/d,,,
1,66-1181361150,"Piso en venta en Calle del Poeta Joan Maragall, 3, Cuatro Caminos - Azca",https://www.fotocasa . 1361150/d/,,,
2,54-103568328,Piso en venta en Cuatro Caminos,https://www.idealist . 103568328/,,,
3,54-103458638,Piso en venta en calle de Pedro Teixeira,https://www.idealist . 103458638/,,,


## Running more complex queries

In [8]:
params = {

    'address': 'Av de Brasil 3, Madrid',     
    'area': '100,200',
    'is_active': 1,
    'operation_type_id': 1,
    'distance': 1000,

    # in order to get a finer scoring and sort by similarity
    'target_area': 150,
    'target_floor': 3,
    'target_baths': 2,
    'target_rooms': 3,
    'target_property_type': 1, # penthouses
    'target_has_elevator': 1,
    'target_has_storage': 1,
    'target_is_exterior': 1,
    'target_has_terrace': 1,
    'target_build_status': 1,
    
    'sort_by': 'similarity',
    'ascending': 0,

}

search = Search().request(**params)

search.df[:3]

Total: 147 - Page: 1/5


,_id,title,is_active,action_id,action_name,usage_id,usage_name,comparison_type,operation_type_id,operation_type_name,...,score_build_status_id,score_has_elevator,score_has_storage,score_has_terrace,score_has_air_conditioner,score_has_garage,score_has_pool,score_is_exterior,score_property_type_id,score_distance
0,54-105013400,Ático en venta en calle de Marceliano Santa María,1,2,Old,1,Residential,1,1,Venta,...,100,100,100,100,None,None,None,100,100.0,88.947216
1,54-104894877,"Ático en venta en calle de Orense, 29",1,2,Old,1,Residential,1,1,Venta,...,100,100,0,100,None,None,None,100,100.0,97.597347
2,54-104594182,Ático en venta en Cuatro Caminos,1,2,Old,1,Residential,1,1,Venta,...,100,100,0,100,None,None,None,100,100.0,100.000000


In [9]:
show_images(search.df[:3])

,_id,title,url,img1,img2,img3
0,54-105013400,Ático en venta en calle de Marceliano Santa María,https://www.idealist . 105013400/,,,
1,54-104894877,"Ático en venta en calle de Orense, 29",https://www.idealist . 104894877/,,,
2,54-104594182,Ático en venta en Cuatro Caminos,https://www.idealist . 104594182/,,,


## Checking on similarity scores

In [10]:
search.df.describe().score

count    30.000000
mean     79.149766
std       4.442971
min      74.961977
25%      75.731935
50%      77.932430
75%      81.028217
max      92.694178
Name: score, dtype: float64

## Checking on uDA's Valuation (AVM)

In [11]:
search.df.valuation_sale.describe().round()

count         30.0
mean      762802.0
std       191231.0
min       464188.0
25%       623410.0
50%       760730.0
75%       877775.0
max      1413220.0
Name: valuation_sale, dtype: float64

## Checking on other interesting indicators

In [12]:
search = Search().request(
    address="Villaverde, Madrid", 
    price='0,100000',
    area='50,150',
    operation_type_id=1, # sale
    gross_yield_asking_price = '5,12',
    target_area=100,
    is_active=1, # published
    sort_by='gross_yield_asking_price', 
    ascending=0,
    distance=1000,
)

maindrivers_zone = [c for c in search.df.columns if 'md_' in c]

others = ['_id', 'title', 'admin3', 'admin4', 'admin5', 'url', 'last_price_with_correction', 'valuation_sale', 'time_on_market_sale','time_on_market_rent', 'price_gap', 'gross_yield_asking_price']

search.df[others]

Total: 8 - Page: 1/1


,_id,title,admin3,admin4,admin5,url,last_price_with_correction,valuation_sale,time_on_market_sale,time_on_market_rent,price_gap,gross_yield_asking_price
0,54-105064118,Piso en venta en calle Calamina,Madrid,Villaverde,San Andrés,https://www.idealista.com/inmueble/105064118/,77710.08,125524.78,8.0,5.0,44.73,11.52
1,54-102609410,Piso en venta en Villaverde Alto,Madrid,Villaverde,San Andrés,https://www.idealista.com/inmueble/102609410/,73382.40,111473.33,8.0,5.0,36.11,11.18
2,54-104644077,Piso en venta en paseo Talleres,Madrid,Villaverde,San Andrés,https://www.idealista.com/inmueble/104644077/,80192.00,136588.42,8.0,5.0,52.61,11.01
3,54-104885544,Piso en venta en calle de Albino Hernández Lázaro,Madrid,Villaverde,San Andrés,https://www.idealista.com/inmueble/104885544/,77056.00,118283.88,9.0,3.0,37.54,10.57
4,54-102978094,Piso en venta en Los Ángeles,Madrid,Villaverde,San Andrés,https://www.idealista.com/inmueble/102978094/,76070.40,98018.89,9.0,3.0,15.45,10.25
5,54-104876517,Piso en venta en Villaverde Alto,Madrid,Villaverde,San Andrés,https://www.idealista.com/inmueble/104876517/,80640.00,94552.85,9.0,3.0,5.06,7.79
6,54-101263159,Dúplex en venta en calle de la Fuente,Madrid,Villaverde,San Andrés,https://www.idealista.com/inmueble/101263159/,85120.00,102475.43,9.0,3.0,7.87,7.77
7,54-101263161,Dúplex en venta en calle de la Fuente,Madrid,Villaverde,San Andrés,https://www.idealista.com/inmueble/101263161/,86912.00,102475.43,9.0,3.0,5.64,7.61


## Checking on cadastral matchs

In [13]:
search = Search().request(
    address="Calle de Huesca 27, Madrid", 
    operation_type_id=1, # sale
    is_active=1, # published
    sort_by='similarity', 
    ascending=0
)

def read_match(x):
    print(f'\n{x._id} - {x.title} {x.url}')
    if match:=x.cadastral_match:
        match = sorted(match, key = lambda x: -x['score'])
        for c in match[:3]:
            print('\t', c.get('address'), c.get('floor'), c.get('cadastral_bi'), c.get('area_total_con_elementos_comunes'), c.get('score_10'))
        print('\t...')
        return
    print('\t...')
        
_ = search.df[:5].apply(lambda x: read_match(x), axis=1)

Total: 118 - Page: 1/4

54-104965509 - Piso en venta en calle de Huesca, 19 https://www.idealista.com/inmueble/104965509/
	 CALLE HUESCA 19, MADRID (MADRID) 2 0890408VK4709B0008WD 85.0 7.0
	 CALLE HUESCA 19, MADRID (MADRID) 2 0890408VK4709B0009EF 79.0 7.0
	 CALLE HUESCA 19, MADRID (MADRID) 2 0890408VK4709B0007QS 71.0 7.0
	...

54-104869914 - Piso en venta en Cuzco-Castillejos https://www.idealista.com/inmueble/104869914/
	 CALLE ROBLE 1, MADRID (MADRID) 6 0994803VK4709D0029MM 146.0 0.0
	 CALLE SAN GERMAN 57, MADRID (MADRID) 6 0789101VK4708H0214MD 157.0 0.0
	 CALLE ROBLE 3, MADRID (MADRID) 6 0994803VK4709D0060LL 150.0 0.0
	...

66-1182650959 - Piso en venta en Calle Hermano Gárate, Castillejos - Cuzco https://www.fotocasa.es/es/comprar/vivienda/madrid-capital/calefaccion-ascensor/182650959/d
	...

54-104072229 - Piso en venta en calle de San Germán https://www.idealista.com/inmueble/104072229/
	 CALLE SAN GERMAN 37, MADRID (MADRID) 2 0888210VK4708H0026WH 109.0 4.0
	 CALLE SAN GERMAN 37,

## Checking on closings matchs

In [14]:
search = Search().request(
    address="calle huescsa 27, Madrid", 
    operation_type_id=1, # sale
    is_active=0, # published
    target_area=100,
    sort_by='score', 
    ascending=0
)

def read_match(x):
    print(f'\n{x._id} - {x.title} {x.score}')
    if match:=x.closing_match:
        match = sorted(match, key = lambda x: -x['score'])
        for c in match[:3]:
            print('\t', 'id:', c.get('clo_id_uda'), 'score:', c.get('score'))
        print('\t...')
        return
    print('\t...')
        
_ = search.df[:10].apply(lambda x: read_match(x), axis=1)

Total: 250 - Page: 1/9

66-1178501558 - Piso en venta en Castillejos - Cuzco 100.0
	...

54-103078289 - Piso en venta en calle de la Infanta Mercedes, 34 100.0
	 id: 41-5cfc122de6a582b7c22de712570c6c88_1 score: 5
	 id: 41-d0b63cceaf50d7644ea32e6ee1d63f95_1 score: 5
	 id: 41-6b9fc394b993ef676789e664536e3851_1 score: 5
	...

54-102935025 - Piso en venta en calle Hermano Gárate 100.0
	...

54-100804924 - Piso en venta en calle Hermano Gárate, 1 100.0
	 id: 41-711fe67fa356eabd2abbda81f622e25d_1 score: 0
	 id: 41-9719b5e72597353a7b76e007675e32cc_1 score: 0
	 id: 41-4b88862b53c6a236c2a27abca2260b53_1 score: 0
	...

54-101584916 - Dúplex en venta en Alonso Castrillo, 19 100.0
	 id: 41-c26b6334135ac16f497c05860868219f_1 score: 0
	 id: 41-aa2d1d559b49d62c73d8adb2617938a5_1 score: 0
	 id: 41-0267ac6c82dd86428654aec714862a56_1 score: 0
	...

54-102492870 - Piso en venta en calle de la Infanta Mercedes 100.0
	...

66-1165345109 - Piso en venta en Calle del Marqués de Leis 16 Castillejos - Cuzco 10

## Getting details on a single or multiple records

In [15]:
search = Search().request(
    id_uda="54-103078289,41-5cfc122de6a582b7c22de712570c6c88_1"
)

search.df.T.dropna().to_dict()

Total: 2 - Page: 1/1


{0: {'_id': '41-5cfc122de6a582b7c22de712570c6c88_1',
  'is_active': 0,
  'usage_id': 1,
  'usage_name': 'Residential',
  'comparison_type': 2,
  'operation_type_id': 1,
  'operation_type_name': 'Venta',
  'last_price': 253000.0,
  'last_pm2': 2842.7,
  'address': 'INFANTA MERCEDES, 34',
  'location': {'coordinates': [-3.6976080847, 40.4571369517], 'type': 'Point'},
  'similarity': 2,
  'property_type_id': 4,
  'property_type_name': 'Piso',
  'typology_type_id': 1,
  'typology_type_name': 'Residencial',
  'construction_type_id': 2,
  'construction_type_name': 'Segunda mano',
  'build_status_id': 1,
  'build_status_name': 'Good condition',
  'has_elevator': 1,
  'is_duplicate': 0,
  'is_vpo': 0,
  'cat_area': 2,
  'category_id': 2,
  'area': 89.0,
  'is_accurate_address': 1,
  'admin0': 'Spain',
  'admin1': 'Comunidad de Madrid',
  'admin2': 'Madrid',
  'admin3': 'Madrid',
  'admin4': 'Tetuán',
  'admin5': 'Castillejos',
  'admin6': 'Sección 060',
  'id_admin0': 1,
  'id_admin1': 14,
  '